In [3]:
import requests
import datetime
from urllib.parse import urlencode

In [4]:
import base64

In [5]:
client_id = '0fbfa1d44348460d9d15a2f434f22527'
client_secret = '4b12f375aed54ab4b50420f1ec381cdb'

In [15]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expires= True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    
    def __init__(self,client_id, client_secret, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" 
        }
    
    def get_token_data(self): 
        return {
            "grant_type": "client_credentials"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)        
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client")
        data = r.json()
        now = datetime.datetime.now()
        access_token= data['access_token']
        expires_in= data['expires_in']  #seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token=access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires<now: 
            self.perform_auth()
            return self.get_access_token()
        elif token == None:        
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self,lookup_id, resource_type='album', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_headers()   
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
        
                
    def get_album(self,_id):
        return self.get_resource(_id, resource_type='album')
    
    def get_artist(self,_id):
        return self.get_resource(_id, resource_type='artists')
    
    def get_track(self,_id):
        return self.get_resource(_id,resource_type='tracks')
        
    
    def search(self, query, search_type='artist'):
        headers = self.get_resource_headers()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q":query,"type":search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()

In [16]:
spotify = SpotifyAPI(client_id, client_secret)

In [17]:
spotify.search("Muchachos", search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Muchachos&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/60nua3AsVSfADZtg5Hdz3W'},
       'href': 'https://api.spotify.com/v1/artists/60nua3AsVSfADZtg5Hdz3W',
       'id': '60nua3AsVSfADZtg5Hdz3W',
       'name': 'La Mosca Tse-Tse',
       'type': 'artist',
       'uri': 'spotify:artist:60nua3AsVSfADZtg5Hdz3W'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',


In [18]:
spotify.get_artist("60nua3AsVSfADZtg5Hdz3W")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/60nua3AsVSfADZtg5Hdz3W'},
 'followers': {'href': None, 'total': 451381},
 'genres': ['argentine rock',
  'latin alternative',
  'latin pop',
  'latin rock',
  'rock en espanol',
  'ska argentino'],
 'href': 'https://api.spotify.com/v1/artists/60nua3AsVSfADZtg5Hdz3W',
 'id': '60nua3AsVSfADZtg5Hdz3W',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb1faececdec07053538cd9e76',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051741faececdec07053538cd9e76',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1781faececdec07053538cd9e76',
   'width': 160}],
 'name': 'La Mosca Tse-Tse',
 'popularity': 67,
 'type': 'artist',
 'uri': 'spotify:artist:60nua3AsVSfADZtg5Hdz3W'}

In [19]:
spotify.get_track("79d6fVzRM6DRYxR302AJFY")

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/60nua3AsVSfADZtg5Hdz3W'},
    'href': 'https://api.spotify.com/v1/artists/60nua3AsVSfADZtg5Hdz3W',
    'id': '60nua3AsVSfADZtg5Hdz3W',
    'name': 'La Mosca Tse-Tse',
    'type': 'artist',
    'uri': 'spotify:artist:60nua3AsVSfADZtg5Hdz3W'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'ET',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   